In [ ]:
!pip install python-roborock
!pip install google-cloud-aiplatform[adk,agent_engines]


In [15]:
from dotenv import load_dotenv

load_dotenv()


True

In [16]:

from google.adk.agents import Agent
import vertexai

# Import Roborock libraries
from roborock import HomeDataProduct, DeviceData, RoborockCommand
from roborock.version_1_apis import RoborockMqttClientV1, RoborockLocalClientV1
from roborock.web_api import RoborockApiClient

import os

# import all functions from the agent.py in the local directory
import agent


In [27]:
# Global variables to hold the MQTT client and device
mqtt_client = None
device = None

# Helper function to get environment variables
def get_env_var(key):
    value = os.getenv(key)
    if value is None:
        raise ValueError(f"Environment variable '{key}' not found.")
    return value

# Login to Roborock (will only run if mqtt_client is None)
async def ensure_login():
    global mqtt_client
    global device
    if mqtt_client is None:
        try:
            web_api = RoborockApiClient(username=get_env_var('ROBOROCK_USERNAME'))
            user_data = await web_api.pass_login(password=get_env_var('ROBOROCK_PASSWORD'))
            home_data = await web_api.get_home_data_v2(user_data)
            device_data = home_data.devices[0]
            product_info: dict[str, HomeDataProduct] = {
                product.id: product for product in home_data.products
            }
            device = DeviceData(device_data, product_info[device_data.product_id].model)
            mqtt_client = RoborockMqttClientV1(user_data, device)
            await mqtt_client.async_connect()
            print("Roborock login successful.")
            return True
        except Exception as e:
            print(f"Roborock login failed: {e}")
            mqtt_client = None
            device = None
            return False
    return True

# Resets Roborock login and session
async def reset_connection():
    global mqtt_client
    global device
    if mqtt_client:
        try:
            await mqtt_client.async_disconnect()
            print("MQTT client disconnected.")
        except Exception as e:
            print(f"Error disconnecting MQTT client: {e}")
        finally:
            mqtt_client = None
            device = None
            print("Roborock connection reset.")

# Get Roborock status
async def get_status():
    if not await ensure_login():
        return {"error": "Not logged in to Roborock."}
    try:
        status = await mqtt_client.get_status()
        print("Current Status:")
        print(status)
        return {
            "state": status.state_name,
            "battery": status.battery,
            "clean_time": status.clean_time,
            "clean_area": status.square_meter_clean_area,
            "error": status.error_code_name,
            "fan_speed": status.fan_power_name,
            "mop_mode": status.mop_mode_name,
            "docked": status.state_name == "charging"
        }
    except Exception as e:
        print(f"Error getting status: {e}")
        await reset_connection()
        return {"error": f"Error getting status: {e}. Connection reset."}

# Send basic Roborock commands that don't have parameters
async def send_basic_command(command: str) -> str:
    if not await ensure_login():
        return {"error": "Not logged in to Roborock."}
    try:
        await mqtt_client.send_command(command)
        print(f"Command sent: {command}")
        return {"result": f"Command {command} sent successfully."}
    except Exception as e:
        print(f"Error sending {command}: {e}")
        await reset_connection()
        return {"error": f"Error sending {command}: {e}. Connection reset."}

# cleans a specific room also known as segment. To Do is to make this dynamic based upon desired segment from instructions mapping in the Agent definition below. 
async def app_segment_clean(segment_number: dict) -> str:
    if not await ensure_login():
        return {"error": "Not logged in to Roborock."}
    command = "app_segment_clean"
    try:
        segment = await mqtt_client.send_command(command, [{"segments": segment_number, "repeat": 1}])
        print(f"Command sent: {command}")
        return segment
    except Exception as e:
        print(f"Error sending {command}: {e}")
        await reset_connection()
        return {"error": f"Error sending {command}: {e}. Connection reset."}


In [28]:
# root agent definition
root_agent =  Agent(
    name="Roborock_Agent", # ensure no spaces here
    # model="gemini-2.0-flash-exp", # for live api
    model="gemini-2.0-flash",
    description="Agent to control and get status of your Roborock vacuum",
    # natural language instruction set which explains to the agent its capabilities and how to operate
    instruction="""I can control and get the status of your Roborock vacuum.
        I can handle the following commands:
        - get_status (this command gets the current status of the Roborock)
        when the above command is needed, call the get_status function
        
        Additional commands.  Use the send_basic_command function for these commands
        - app_charge (this command sends the Roborock back to the dock)
        - app_start_wash (this command starts the washing of the mop while docked)
        - app_stop_wash (this command stops the washing of the mop while docked)
        - app_start (this command starts vacuuming and mopping job)
        - app_stop (this command stops the vacuuming and mopping job)
        - app_pause (this command pauses the vacuuming and mopping job)
        - app_start_collect_dust (this command starts emptying the dust bin)
        - app_stop_collect_dust (this command stops emptying the dust bin)
        - get_room_mapping (gets a list of the rooms in a map)

        when the above commands are needed, call send_basic_command(command).  Examples:
        - for app_start_wash call send_basic_command("app_start_wash")
        - for app_stop_wash call send_bacic_command("app_stop_wash")

        Additional commands:
        - app_segment_clean (this command starts cleaning rooms or segments) 
        - for this command, use the function app_segment_clean function
        - when using this command, you must pass a segment number from the mapping below as a list of integers.  
        - For example, for a request to clean Bedroom4, you would call:
        app_segment_clean([16])
        - if multiple rooms are specified,
        app_segment_clean([16,18])


        Segment mapping:
        16 = Bedroom4
        17 = Balcony
        18 = Bedroom3
        19 = Bathroom
        20 = Hallway
        21 = Kitchen
        22 = Dining Room
        23 = Entryway
        24 = Bedroom1
        25 = Bedroom2
        26 = Living Room

        ***Finally, format the status as a nice table***

        """,
    # tells the agent what tools (function names from above) it has access to. The agent uses the instructions above to understand how and when to use these tools. 
    tools=[
        get_status,
        send_basic_command,
        app_segment_clean,
    ],
)

In [25]:
# Global variables to hold the MQTT client and device
mqtt_client = None
device = None

# Helper function to get environment variables
def get_env_var(key):
    value = os.getenv(key)
    if value is None:
        raise ValueError(f"Environment variable '{key}' not found.")
    return value

project_id=get_env_var("GOOGLE_CLOUD_PROJECT")
staging_bucket=get_env_var("GOOGLE_CLOUD_STORAGE_BUCKET")
location=get_env_var("GOOGLE_CLOUD_LOCATION")

# initialitze vertexai
vertexai.init(
    project=project_id,
    location=location,
    staging_bucket=staging_bucket,
)


# root agent definition
root_agent = agent.root_agent

In [29]:
from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

In [24]:
# In a Jupyter cell
!gcloud auth list
!gcloud config list project
!echo $GOOGLE_APPLICATION_CREDENTIALS # On Linux/macOS
# !echo %GOOGLE_APPLICATION_CREDENTIALS% # On Windows CMD
# !Get-ChildItem Env:GOOGLE_APPLICATION_CREDENTIALS # On Windows PowerShell


            Credentialed Accounts
ACTIVE  ACCOUNT
*       admin@jeffdahan.altostrat.com
        jeffdahan@stevephillips.altostrat.com
        jeffreydahan@gmail.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`

[core]
project = fine-tuning-webinar

Your active configuration is: [default]



In [ ]:
#!gcloud auth application-default login --project {project_id}
#!gcloud config list project


from google.colab import auth
auth.authenticate_user(project_id=PROJECT_ID)

from vertexai import agent_engines

remote_app = agent_engines.create(
    agent_engine=root_agent,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]",
        "python-roborock"   
    ],
    display_name="Roborock Agent 1",
    description="The Roborock Agent can control and get the status for the Roborock Vacuum",
    #gcs_dir_name=get_env_var("GOOGLE_CLOUD_STORAGE_BUCKET")
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'pydantic': '2.11.3', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.90.0'}
The following requirements are missing: {'pydantic', 'cloudpickle'}
The following requirements are appended: {'cloudpickle==3.1.1', 'pydantic==2.11.3'}
The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'python-roborock', 'cloudpickle==3.1.1', 'pydantic==2.11.3']
Using bucket adk-agent-engine-staging
Wrote to gs://adk-agent-engine-staging/agent_engine/agent_engine.pkl
Writing to gs://adk-agent-engine-staging/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://adk-agent-engine-staging/agent_engine/dependencies.tar.gz


ServiceUnavailable: 503 Getting metadata from plugin failed with error: Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.

In [30]:
remote_app.resource_name

NameError: name 'remote_app' is not defined

In [28]:
remote_session = remote_app.create_session(user_id="u_456")
remote_session

{'last_update_time': 1747937006.029384,
 'id': '7899221937186930688',
 'app_name': '6774583719937179648',
 'state': {},
 'events': [],
 'user_id': 'u_456'}

In [29]:
remote_app.list_sessions(user_id="u_456")

{'sessions': [{'app_name': '6774583719937179648',
   'id': '7899221937186930688',
   'last_update_time': 1747937006.029384,
   'events': [],
   'state': {},
   'user_id': 'u_456'}]}

In [30]:
remote_app.get_session(user_id="u_456", session_id=remote_session["id"])

{'app_name': '6774583719937179648',
 'id': '7899221937186930688',
 'last_update_time': 1747937006.029384,
 'events': [],
 'state': {},
 'user_id': 'u_456'}

In [31]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="get the status",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-5fd81692-ab97-434b-9e4b-80556bc71f1d', 'args': {}, 'name': 'get_status'}}], 'role': 'model'}, 'invocation_id': 'e-ef1b446f-cfe7-48e1-8d21-8805c7d0879e', 'author': 'Roborock_Agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': 'cUacl69X', 'timestamp': 1747937024.315067}
{'content': {'parts': [{'function_response': {'id': 'adk-5fd81692-ab97-434b-9e4b-80556bc71f1d', 'name': 'get_status', 'response': {'state': 'charging', 'battery': 100, 'clean_time': 425, 'clean_area': 6.4, 'error': 'none', 'fan_speed': 'max', 'mop_mode': 'standard', 'docked': True}}}], 'role': 'user'}, 'invocation_id': 'e-ef1b446f-cfe7-48e1-8d21-8805c7d0879e', 'author': 'Roborock_Agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'M9aQfDKH', 'timestamp': 1747937028.450545}
{'content': {'parts': [{'text': 'The Roborock is currently charging, with a ba

In [ ]:

!curl -X POST \
    -H "Content-Type: application/json" \
    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
    -H "X-Goog-User-Project: fine-tuning-webinar" \
"https://discoveryengine.googleapis.com/v1alpha/projects/fine-tuning-webinar/locations/global/collections/default_collection/engines/argolis-agentspace-jeffd_1747939249860/assistants/default_assistant/agents" \
-d '{ \
        "displayName": "Roborock Agent 1", \
        "description": "This agent can control and get the status of a Roborock Vacuum", \
        "adk_agent_definition": { \
            "tool_settings": { \
                "tool_description": "This agent can control and get the status of a Roborock Vacuum" \
            }, \
            "provisioned_reasoning_engine": { \
                "reasoning_engine":f"{engine_id}" \
            }, \
        } \
}'

{
  "name": "projects/850217602120/locations/global/collections/default_collection/engines/argolis-agentspace-jeffd_1747939249860/assistants/default_assistant/agents/7760300495693784793",
  "displayName": "Roborock Agent 1",
  "description": "This agent can control and get the status of a Roborock Vacuum",
  "adkAgentDefinition": {
    "toolSettings": {
      "toolDescription": "This agent can control and get the status of a Roborock Vacuum"
    },
    "provisionedReasoningEngine": {
      "reasoningEngine": "projects/850217602120/locations/us-central1/reasoningEngines/6774583719937179648"
    }
  },
  "state": "CONFIGURED"
}
